In [2]:
pip install scikeras

In [3]:
pip install keras==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.12.0 which is incompatible.


In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np

In [2]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [3]:
# Normalize pixel values to the range [0, 1] and cast to float32
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [4]:
# Define the neural network architecture for CIFAR-10
def create_model(neurons_per_layer=64, activation_function='relu'):
    model = Sequential([
        Flatten(input_shape=(32, 32, 3)),  # Assuming RGB images
        Dense(neurons_per_layer, activation=activation_function),
        Dense(100, activation='softmax')  # 100 output units for 100 classes in CIFAR-10
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    return model

In [5]:
# Create KerasClassifier for GridSearchCV
model = KerasClassifier(build_fn=create_model, verbose=0)

<ipython-input-5-f6b835687327>:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [6]:
# Define hyperparameters to tune for CIFAR-10
param_grid = {
    'neurons_per_layer': [64, 128, 256],
    'activation_function': ['relu', 'tanh', 'sigmoid']
}

In [7]:
# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=2)
grid_result = grid_search.fit(x_train, y_train)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END .....activation_function=relu, neurons_per_layer=64; total time=   8.9s
[CV] END .....activation_function=relu, neurons_per_layer=64; total time=   9.1s
[CV] END .....activation_function=relu, neurons_per_layer=64; total time=   8.9s
[CV] END ....activation_function=relu, neurons_per_layer=128; total time=  16.9s
[CV] END ....activation_function=relu, neurons_per_layer=128; total time=  14.5s
[CV] END ....activation_function=relu, neurons_per_layer=128; total time=  10.8s
[CV] END ....activation_function=relu, neurons_per_layer=256; total time=  16.9s
[CV] END ....activation_function=relu, neurons_per_layer=256; total time=  24.3s
[CV] END ....activation_function=relu, neurons_per_layer=256; total time=  25.5s
[CV] END .....activation_function=tanh, neurons_per_layer=64; total time=   9.5s
[CV] END .....activation_function=tanh, neurons_per_layer=64; total time=   8.9s
[CV] END .....activation_function=tanh, neurons_p

In [8]:
# Summarize results
print("Overall Results:")
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Overall Results:
Best: 0.352400 using {'activation_function': 'relu', 'neurons_per_layer': 128}
0.344120 (0.011509) with: {'activation_function': 'relu', 'neurons_per_layer': 64}
0.352400 (0.008177) with: {'activation_function': 'relu', 'neurons_per_layer': 128}
0.350280 (0.000755) with: {'activation_function': 'relu', 'neurons_per_layer': 256}
0.301760 (0.015706) with: {'activation_function': 'tanh', 'neurons_per_layer': 64}
0.300740 (0.010954) with: {'activation_function': 'tanh', 'neurons_per_layer': 128}
0.292420 (0.017267) with: {'activation_function': 'tanh', 'neurons_per_layer': 256}
0.310200 (0.003291) with: {'activation_function': 'sigmoid', 'neurons_per_layer': 64}
0.339260 (0.006484) with: {'activation_function': 'sigmoid', 'neurons_per_layer': 128}
0.338740 (0.011423) with: {'activation_function': 'sigmoid', 'neurons_per_layer': 256}


In [9]:
# Separate results for each activation function
activation_functions = ['relu', 'tanh', 'sigmoid']
for activation_function in activation_functions:
    activation_results = [(mean, std, param) for mean, std, param in zip(means, stds, params) if param['activation_function'] == activation_function]
    print(f"\nResults for {activation_function.capitalize()} activation:")
    for mean, std, param in activation_results:
        print("%f (%f) with: %r" % (mean, std, param))


Results for Relu activation:
0.344120 (0.011509) with: {'activation_function': 'relu', 'neurons_per_layer': 64}
0.352400 (0.008177) with: {'activation_function': 'relu', 'neurons_per_layer': 128}
0.350280 (0.000755) with: {'activation_function': 'relu', 'neurons_per_layer': 256}

Results for Tanh activation:
0.301760 (0.015706) with: {'activation_function': 'tanh', 'neurons_per_layer': 64}
0.300740 (0.010954) with: {'activation_function': 'tanh', 'neurons_per_layer': 128}
0.292420 (0.017267) with: {'activation_function': 'tanh', 'neurons_per_layer': 256}

Results for Sigmoid activation:
0.310200 (0.003291) with: {'activation_function': 'sigmoid', 'neurons_per_layer': 64}
0.339260 (0.006484) with: {'activation_function': 'sigmoid', 'neurons_per_layer': 128}
0.338740 (0.011423) with: {'activation_function': 'sigmoid', 'neurons_per_layer': 256}


In [10]:
# Evaluate the best model
test_acc = grid_search.score(x_test, y_test)
print('\nTest accuracy of the best model:', test_acc)


Test accuracy of the best model: 0.3698999881744385
